# TensorIOT Data Assignment 2

## Submitted by : Deepansh Singh
### Email - deepanshsingh8@gmail.com

Data Assignment 2

·       Download the data files from here - http://jmcauley.ucsd.edu/data/amazon/links.html

·       Apache spark tools locally and necessary tools

·       Download a review file with a million reviews.

·       Using Jupiter notebook create a program to read the million reviews and get the following.

·       transform date to MM-DD-YYYY format.

·       Save the data into a table (postgres/sql server)

·       Save the output as a Parquet file.

·       Upload code to Github and complete Readme.md which anyone can understand.

·       Send Github link to HR

### Method - 1 for reading Data

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd

# Initialize a Spark session
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('AmazonReview - 2') \
    .getOrCreate()

# Specify the path to your JSON file
json_file_path = "Grocery_and_Gourmet_Food.json"

# Read the JSON file into a Spark DataFrame
spark_df = spark.read.json(json_file_path)

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = spark_df.toPandas()

# Show the first few rows of the Pandas DataFrame
print(pandas_df.head())

### Method - 2 for reading data

In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Grocery_and_Gourmet_Food.json.gz')

## Exploratory analysis

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.drop(['vote', 'style', 'image'], axis = 1, inplace = True)

In [ ]:
df['asin'].value_counts()

# Converting reviewTime to date MM-DD-YYYY format.

In [ ]:
df['reviewDate'] = pd.to_datetime(df['unixReviewTime'], unit='s').dt.strftime('%m-%d-%Y')

In [ ]:
df.drop(['reviewTime', 'unixReviewTime'], axis = 1, inplace = True)

In [ ]:
import mysql.connector

connection = mysql.connector.connect(
    user='Deepanshsingh8',
    password='B*************4',
    host='localhost',
    database='TensorIOT'
)

In [ ]:
df.to_sql(name='Groceries', con=connection, if_exists='replace', index=False)

In [ ]:
connection.commit()
connection.close()

# Saving to Parquet file
## Further Bigram and Trigram analysis - but it is out of scope for this assignment

In [ ]:
df.to_parquet('Groceries_and_Gourmet_Food-2.parquet', index=False)

In [ ]:
df